# Goal 1

In this script, I will show the full end-to-end process including making tuning records from the raw dataset.

In many cases, the customer has only raw datasets instead of formalized training set. So, 

1. I will make formalized training dataset efficiently through LLM, 

2. And, Basically LLMs are based on statistics, so I will apply the following input/output data format design.

>  - Input Data Formatting : For example, all input dataset should include the following template - "As a xxx role, we need to answer the following question - ${question}". 
>  - Input Data Duplicating : There is no statistics/evidence for how much impacted duplicated input data will be used. For example 
>
>>      Record 1 : { "input" : "As a banker role, which accounts of xxx bank would be the most beneficial for a foreigner with L1 Visa ?", "Answer : It's the xxx account type."  }
>>      Record 2 : { "input" : "As a banker role, which accounts of xxx bank would be the most beneficial for a foreigner with L1 Visa ?", "Answer : It's the xxx account type."  } -- Duplicated records.
  
3. But the duplicated records could make wrong sub-optimal parameters for the task, we will add some general corpos to enhance 'prefix' layer. 
  In this whitepaper (https://arxiv.org/pdf/2101.00190.pdf), the prefix PEFT uses thw following datasets to optimize the prefix parameters (E2E- 50K, WebNLG - 22K). 

4. I will use prompt template to increase correctness of PEFT model. 

> PROMPT : 'As a banker role, ${question}'

# Goal 2

In this script, how to refine raw dataset such like PDF files. 

During processing PDF files, you can meet the following issues. 

1. Some exceptional phrases (footer/header/numbering) could be added into the context

2. Some phrases are splited by pages. 

3. Pages with two or three columns are often analyzed incorrectly.

4. ETC...

In this script, I will try to solve Issue #1, Issue #2 with LLMs.



## 1. Making Formalized Training Set with more Large LLM

LargeLLM - such like Bard, ChatGPT3.5/4 can make the input/output pairs with raw dataset such like the following. 
(In this example, I will use the Robinhood's account descriptions - https://robinhood.com/us/en/support/articles/robinhood-accounts/)

> Prompt : "As a robinhood counseller, make four questions and answers related with the ${context}"
>
>> Context : Margin accounts - Trade with unsettled funds from stock and option sales. You don’t have to wait for funds to settle (2 trading days for stocks, 1 for options).
>>
>> Access margin investing. If eligible, you can enable margin investing and get more buying power.
>>
>> Access Level 3 options trading. If eligible, you can trade multi-leg options, like spreads.
>>
>> Pattern day trading regulations. If you make 4 or more day trades within 5 trading days in a margin account, you may be flagged as a pattern day trader. Review Pattern day trading for more information.

The quality of these QnA pairs are good. For example. 

> Q1: What is a margin account?
> 
> A1: A margin account is a type of brokerage account that allows investors to trade with borrowed funds, also known as margin, based on the value of their existing securities.
> 
> 
> Q2: How does trading with unsettled funds work in a margin account?
> 
> A2: In a margin account, you can trade with unsettled funds from stock and option sales. This means you don't have to wait for the funds to settle before making new trades.
> 
> 
> Q3: How long does it typically take for funds to settle in a margin account?
> 
> A3: For stocks, it usually takes 2 trading days for funds to settle, while for options, it takes 1 trading day.
> 
> 
> Q4: What is margin investing, and how can I access it?
> 
> A4: Margin investing allows you to leverage your existing securities to increase your buying power. If you are eligible, you can enable margin investing through your brokerage account.
> 

So, we can convert these results into formalized training sets. 

## 2. Formalized Splitter

But What **you have to worry about** is 'context length'. 

In many moder LLMs has its own context length limitations. So we need to split the raw context into small context-awared phrases. This issue is mentioned in Goal# 2.

So I have to make appropriate splitter with LLM at first.

In [1]:
## There is no official Bard's API endpoint. So we will use hand-made wrapper class. 

#! pip3 install --upgrade GoogleBard
#! pip3 install langchain typing_extensions==4.5.0
#! pip3 install pypdf
#! pip3 install chromadb
#! pip3 install pydantic -U

# Instead of using Bard, I replaced it with openAI for trial.

In [2]:
from langchain.document_loaders import PyPDFLoader # for loading the pdf


I choose a Non-English content for multi-lingual cases. 

The below content(samsungpay appcard document) can be downloaded from Hyundai Card site. (https://www.hyundaicard.com/doc/samsungpay_appcard_existing.pdf)

In [3]:
pdf_path = "../resources/samsungpay_appcard_existing.pdf"
loader = PyPDFLoader(pdf_path)

documents = loader.load()

for document in documents:
    print (document)

page_content='제 1 장 총칙\n제 1 조 (목적)\n이 약관은 현대카드 주식회사(이하 “회사”라 합니다.)가 “가입 고객”에게 제공하는 “삼성페이 \n용 앱카드 결제 서비스” 의 이용과 관련하여 필요한 회원과 회사간의 권리, 의무 및 책임사항, \n서비스 이용에 따른 이용조건 및 절차 등 기타 제반 사항을 규정함을 목적으로 합니다.\n제 2 조 (용어의 정의)\n①"삼성페이 용 앱카드 결제 서비스"(이하 "서비스"라 합니다.)란 "가입 고객"이 본인의 "모바\n일 기기”에 설치된 “전용 어플리케이션”을 통해 오프라인 및 온라인 가맹점에서 결제승인절차\n를 수행하는 서비스를 말합니다.\n②“가입 고객”이란 “모바일 기기”에 “전용 어플리케이션”을 설치하고 “대상카드”의 카드번호, \n카드비밀번호, 카드고유확인번호(CVC), 휴대전화번호, 생년월일(이하 “카드번호 등”이라 합니\n다.)를 등록하여 “서비스” 이용을 신청하고, “회사”의 인증 및 승낙을 받아 “회사”와 “서비스” \n이용 계약을 체결한 고객을 말합니다.\n③“대상카드”란 “서비스”를 적용하고자 하는 카드로서, “회사”가 발급한 신용카드, 체크카드를 \n말합니다.\n④“휴대폰 인증”이란 “가입 고객”이 본인 명의로 3G 및 4G이동통신망에 가입한 “모바일 기기”\n로 수신 받은 인증번호를 입력하도록 하는 방식으로 “가입 고객”의 본인확인을 하는 인증 절\n차를 말합니다.\n⑤“앱카드 결제번호”란 “카드번호 등”을 대신하여 “가맹점”에 제시하는 것으로 NFC, QR코드, \n바코드 등의 형태로 제공되는 일회용 카드번호 및 이미지 일체를 말합니다.\n⑥“결제 비밀번호”란 서비스 부정사용 및 부정 접근을 방지하기 위하여 사용되는 회원 인증 \n암호로 “서비스” 이용을 위하여 “가입 고객”이 별도로 설정한 서비스 비밀번호(숫자6자리)를 \n말합니다.\n⑦“인증정보”란 서비스 화면에서 정하는 인증절차에 이용되는 본인 확인정보를 말하며, “결제 \n비밀번호”, “지문” 등이 인증정보에 해당합니다.\

In [4]:
SPLITTER_PROMPT_TEMPLATE = """
"As a credit card counseller, remove useless footer/header words and divide the paragraphs into sections with hierarchical section numbers which could be given in context in CSV format. 
Hierarchical Section numbering rule : 제1장 - 제1조 - ① - 1
CSV format is <hierarchical section number> , <phrase>
Context : last section number : 2-3-3
 4. This card MUST be used by the card owner. A. Some cards can be used in your mobile platform
Result : 
"2-3-4", "4. This card MUST be used by the card owner."
"2-3-4-1", "A. Some cards can be used in your mobile platform"

Context : last section number : {last_section_number} \n {content}
"""

In [5]:
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

def formalized_splitter(document, last_section_number):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages = [{"role": "system", "content" : SPLITTER_PROMPT_TEMPLATE.format(last_section_number=last_section_number, content=document)}]
    )
    return response

# print(len(documents[0].page_content))


In [6]:
import csv
import traceback

phrases = []
analyzed_phrases = ""
skipped_phrases = []
MAX_TRY_COUNT = 2
MIN_ANALYZED_CONTENT_LENGTH = 10
MIN_RESULT_CONTENT_LENGTH = 10

def full_page_splitter(documents):
  last_section_number = ""
  last_phrase_text = ""
  for document in documents:
    phrases_list = []
    analysis_target_text = last_phrase_text + '\n' + document.page_content
    if len(analysis_target_text) < MIN_ANALYZED_CONTENT_LENGTH :
      last_phrase_text = analysis_target_text
      continue
    try_count = 0
    while True:
      try:
        response = formalized_splitter(document=analysis_target_text, last_section_number=last_section_number)
        analyzed_phrases = response['choices'][0]['message']['content']
        print('##' + analyzed_phrases)
        reader = csv.reader(analyzed_phrases.split('\n'), delimiter=',')
        for row in reader:
          if len(row) >= 2:
            section_number = row[0]
            phrase = ' '.join(row[1:])
            phrases_list.append({ 'section' : section_number, 'phrase' : phrase})
        if len(phrases_list) > 1 :
          last_phrase = phrases_list.pop()
          last_section_number = last_phrase['section']
          last_phrase_text = last_phrase['phrase']
        phrases.extend(phrases_list)
        break
      except:
        print('Error Occured')
        traceback.print_exc()
        try_count = try_count + 1
        if try_count >= MAX_TRY_COUNT :
          skipped_phrases.append(analysis_target_text)
          break
  phrases.append(last_phrase)
  return phrases


In [7]:
import pandas as pd
import fastparquet as fp

parquet_file_path = '../resources/splitted.parquet'
# At first time, you have to  unremark the following three lines
full_page_splitter(documents=documents)
df_phrases = pd.DataFrame.from_records(phrases)
df_phrases.to_parquet(parquet_file_path)

# df_phrases = fp.ParquetFile(parquet_file_path).to_pandas()
# df_phrases

##"1-1", "제 1 장 총칙"
"1-1-1", "제 1 조 (목적)"
"1-1-1-1", "이 약관은 현대카드 주식회사(이하 “회사”라 합니다.)가 “가입 고객”에게 제공하는 “삼성페이 용 앱카드 결제 서비스” 의 이용과 관련하여 필요한 회원과 회사간의 권리, 의무 및 책임사항, 서비스 이용에 따른 이용조건 및 절차 등 기타 제반 사항을 규정함을 목적으로 합니다."
"1-1-2", "제 2 조 (용어의 정의)"
"1-1-2-1", "①“삼성페이 용 앱카드 결제 서비스”(이하 “서비스”라 합니다.)란 “가입 고객”이 본인의 “모바일 기기”에 설치된 “전용 어플리케이션”을 통해 오프라인 및 온라인 가맹점에서 결제승인절차를 수행하는 서비스를 말합니다."
"1-1-2-2", "②“가입 고객”이란 “모바일 기기”에 “전용 어플리케이션”을 설치하고 “대상카드”의 카드번호, 카드비밀번호, 카드고유확인번호(CVC), 휴대전화번호, 생년월일(이하 “카드번호 등”이라 합니다.)를 등록하여 “서비스” 이용을 신청하고, “회사”의 인증 및 승낙을 받아 “회사”와 “서비스” 이용 계약을 체결한 고객을 말합니다."
"1-1-2-3", "③“대상카드”란 “서비스”를 적용하고자 하는 카드로서, “회사”가 발급한 신용카드, 체크카드를 말합니다."
"1-1-2-4", "④“휴대폰 인증”이란 “가입 고객”이 본인 명의로 3G 및 4G이동통신망에 가입한 “모바일 기기”로 수신 받은 인증번호를 입력하도록 하는 방식으로 “가입 고객”의 본인확인을 하는 인증 절차를 말합니다."
"1-1-2-5", "⑤“앱카드 결제번호”란 “카드번호 등”을 대신하여 “가맹점”에 제시하는 것으로 NFC, QR코드, 바코드 등의 형태로 제공되는 일회용 카드번호 및 이미지 일체를 말합니다."
"1-1-2-6", "⑥“결제 비밀번호”란 서비스 부정사용 및 부정 접근을 방지하기 위하여 사용되는 회원 인증 암호로 “서비스” 이용을 위하여 “가입 고객”이 별도로 설정한 서비스 비밀번호(숫자6자리)를 말합니다."
"1-1-2-

In [8]:
phrases

[{'section': '1-1', 'phrase': ' "제 1 장 총칙"'},
 {'section': '1-1-1', 'phrase': ' "제 1 조 (목적)"'},
 {'section': '1-1-1-1',
  'phrase': ' "이 약관은 현대카드 주식회사(이하 “회사”라 합니다.)가 “가입 고객”에게 제공하는 “삼성페이 용 앱카드 결제 서비스” 의 이용과 관련하여 필요한 회원과 회사간의 권리  의무 및 책임사항  서비스 이용에 따른 이용조건 및 절차 등 기타 제반 사항을 규정함을 목적으로 합니다."'},
 {'section': '1-1-2', 'phrase': ' "제 2 조 (용어의 정의)"'},
 {'section': '1-1-2-1',
  'phrase': ' "①“삼성페이 용 앱카드 결제 서비스”(이하 “서비스”라 합니다.)란 “가입 고객”이 본인의 “모바일 기기”에 설치된 “전용 어플리케이션”을 통해 오프라인 및 온라인 가맹점에서 결제승인절차를 수행하는 서비스를 말합니다."'},
 {'section': '1-1-2-2',
  'phrase': ' "②“가입 고객”이란 “모바일 기기”에 “전용 어플리케이션”을 설치하고 “대상카드”의 카드번호  카드비밀번호  카드고유확인번호(CVC)  휴대전화번호  생년월일(이하 “카드번호 등”이라 합니다.)를 등록하여 “서비스” 이용을 신청하고  “회사”의 인증 및 승낙을 받아 “회사”와 “서비스” 이용 계약을 체결한 고객을 말합니다."'},
 {'section': '1-1-2-3',
  'phrase': ' "③“대상카드”란 “서비스”를 적용하고자 하는 카드로서  “회사”가 발급한 신용카드  체크카드를 말합니다."'},
 {'section': '1-1-2-4',
  'phrase': ' "④“휴대폰 인증”이란 “가입 고객”이 본인 명의로 3G 및 4G이동통신망에 가입한 “모바일 기기”로 수신 받은 인증번호를 입력하도록 하는 방식으로 “가입 고객”의 본인확인을 하는 인증 절차를 말합니다."'},
 {'se

Ok. You can see the splittd content with appropriate phrases (but wrong section numbering. but it doens't matter)

We will make the QnA pairs with these splitted context.

Let's do it.

In [38]:
MAKING_QNA_PAIR_PROMPT_TEMPLATE = """
As a credit card counseller, make {number_of_qna} QnA pairs based on the $Context with reasoning in CSV format in English.
Example Context : This card will only be used in Mobile App. 
Example Result : 
"question", "reasoning", "answer", 
"Which platform the user can use this card ?", "In the context, it said only mobile app. so mobile app is correct answer", "mobile app platform"

Context : 
{context}
"""

In [39]:
import csv


def formalized_qna_maker(phrases, number_of_qna):
    try_count = 0
    skipped_phrases = ""
    while True:
        qna_pairs = []
        try_count = try_count + 1
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k",
                messages = [{"role": "system", "content" : MAKING_QNA_PAIR_PROMPT_TEMPLATE.format(context=phrases, number_of_qna=number_of_qna)}]
            )
            analyzed_phrases = response['choices'][0]['message']['content']
            print(analyzed_phrases)
            reader = csv.reader(analyzed_phrases.split('\n'), delimiter=',')
            for row in reader:
                if len(row) >= 3:
                    question = row[0]
                    reason = row[1]
                    answer = ' '.join(row[2:])
                    qna_pairs.append({'question': question, 'reason': reason, 'answer': answer})
            break
        except:
            print("Error Occured : {phrase}".format(phrases))
            if try_count >= MAX_TRY_COUNT:
                skipped_phrases = phrases
                break
    return qna_pairs, skipped_phrases

To make context-awared QnAs, we will use hierarchical section number.

All phrases would be reaggregate with sections. 

I will just use two-depth section number.

In [40]:
last_parent_phrase = ""
last_parent_section = ""
last_phrase = ""
last_section = ""
last_aggregate_phrase = ""
aggregate_phrase = ""
total_qna_pairs = []

MIN_PHRASE_SIZE = 100
MAX_PHRASE_SIZE = 3000

NUM_OF_QUESTION_ESTIMATION_SIZE = 30

def has_same_depth(last_section, section):
  last_sections = last_section.split('-')
  sections = section.split('-')
  # zero depth ?
  if len(last_sections) == 0 or len(sections) == 0 :
    return False
  # one dpeth ?
  if len(last_sections) == 1 or len(sections) == 1 :
    return last_sections[0] == sections[0]
  #print(len(last_sections), len(sections), len(last_sections) == 1 | len(sections) == 1)
  return last_sections[1] == sections[1]

def full_qna_maker(phrases):
  skipped_phrases_list = []
  last_aggregate_phrase = ""
  for phrase in phrases:
    section = phrase['section']
    aggregate_phrase = last_aggregate_phrase + '\n' + phrase['phrase']
    if len(aggregate_phrase) > MIN_PHRASE_SIZE:
      if len(aggregate_phrase) > MAX_PHRASE_SIZE:
        num_of_questions = len(last_aggregate_phrase) / NUM_OF_QUESTION_ESTIMATION_SIZE
        qna_pairs, skipped_phrases = formalized_qna_maker(last_aggregate_phrase, num_of_questions)
        total_qna_pairs.extend(qna_pairs)
        skipped_phrases_list.append(skipped_phrases)
        last_aggregate_phrase = phrase['phrase']
        last_section = section
        continue
      elif not has_same_depth(last_section=last_section, section=section):
        num_of_questions = len(last_aggregate_phrase) / NUM_OF_QUESTION_ESTIMATION_SIZE
        qna_pairs, skipped_phrases = formalized_qna_maker(last_aggregate_phrase, num_of_questions)
        total_qna_pairs.extend(qna_pairs)
        skipped_phrases_list.append(skipped_phrases)
        last_aggregate_phrase = phrase['phrase']
        last_section = section
      else:
        last_section = section
        last_aggregate_phrase = aggregate_phrase
    else:
      last_section = section
      last_aggregate_phrase = aggregate_phrase
  num_of_questions = len(last_aggregate_phrase) / NUM_OF_QUESTION_ESTIMATION_SIZE
  qna_pairs, skipped_phrases = formalized_qna_maker(last_aggregate_phrase, num_of_questions)
  total_qna_pairs.extend(qna_pairs)
  skipped_phrases_list.append(skipped_phrases)
  return total_qna_pairs, skipped_phrases_list




  

In [41]:
total_qna_pairs, skipped_phrases = full_qna_maker(phrases=phrases)


"question", "reasoning", "answer"
"What is the purpose of this agreement?", "In the context, it states that this agreement is intended to regulate the rights, obligations, responsibilities, and conditions for using the 'Samsung Pay App Card Payment Service' provided by Hyundai Card Co., Ltd. for its customers.", "The purpose of this agreement is to regulate the rights, obligations, responsibilities, and conditions for using the 'Samsung Pay App Card Payment Service' provided by Hyundai Card Co., Ltd."
"What is the definition of 'Samsung Pay App Card Payment Service'?", "According to the context, 'Samsung Pay App Card Payment Service' refers to the service that allows 'customers' to make payment authorization procedures at offline and online merchants through a dedicated application installed on their 'mobile devices'.", "The 'Samsung Pay App Card Payment Service' refers to the service that allows 'customers' to make payment authorization procedures at offline and online merchants throu

In [42]:
total_qna_pairs

[{'question': 'question', 'reason': ' "reasoning"', 'answer': ' "answer"'},
 {'question': 'What is the purpose of this agreement?',
  'reason': ' "In the context',
  'answer': ' it states that this agreement is intended to regulate the rights  obligations  responsibilities  and conditions for using the \'Samsung Pay App Card Payment Service\' provided by Hyundai Card Co.  Ltd. for its customers."  "The purpose of this agreement is to regulate the rights  obligations  responsibilities  and conditions for using the \'Samsung Pay App Card Payment Service\' provided by Hyundai Card Co.  Ltd."'},
 {'question': "What is the definition of 'Samsung Pay App Card Payment Service'?",
  'reason': ' "According to the context',
  'answer': ' \'Samsung Pay App Card Payment Service\' refers to the service that allows \'customers\' to make payment authorization procedures at offline and online merchants through a dedicated application installed on their \'mobile devices\'."  "The \'Samsung Pay App Card

If you can see the context of Question / Reasoning / Answer, these contents all have missing information. 

It's the product name and role of this answer. 

So I will add this information into input / output training dataset.

I don't have multi ligual model permission not yet. So I will translate all qna pair into English. 

In [43]:
MISSING_INFORMATION_PADDING_TEMPLATE = 'As a hyundaicard credit card counseller, you are answering to the customer for the samsung pay appcard payment service. Question : {question}'

refined_qna_pairs = []

for qna_pair in total_qna_pairs:
    if len(qna_pair['question']) > 10 and len(qna_pair['answer']) > 10:
        refined_qna_pairs.append( { "input_text" : MISSING_INFORMATION_PADDING_TEMPLATE.format(question=qna_pair['question']), "output_text" : qna_pair['reason'] + qna_pair['answer'] })
        
        


In [45]:
#! pip3 install jsonlines

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


125

In [47]:
import jsonlines

trainingset_file_path = "../resources/training_set.jsonl"

with jsonlines.open(trainingset_file_path, mode='w') as writer:
  for item in refined_qna_pairs:
      writer.write(item)
      writer.write(item)
      writer.write(item)
      writer.write(item)
      writer.write(item)


    

Above codes mutiplied original dataset 5 times. 

But in the witepaper(https://arxiv.org/pdf/2101.00190.pdf), they used the following training sets. 

- The WebNLG (Gardent et al., 2017) dataset consists of 22K examples,

- 